# S1W2D7 - 训练循环“八股文”与总结

**今日任务：**巩固与默写

今天的学习分为两部分：
1. **实战挑战**：“白板默写训练八股文
2. **理论补完**：深入理解`model.train()`和`model.eval()`的区别

## 核心挑战：”白板“默写挑战

尝试**从头到尾**、**仅凭记忆**写出一个完整的MNIST训练脚本

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

def get_data_loaders(batch_size=64, root='../../data'):
    # 定义数组转换
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # 下载并加载训练数据
    train_dataset = datasets.MNIST(
        root=root,
        train=True,
        download=True,
        transform=transform
    )

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

In [ ]:
# 主脚本

import torch
import torch.nn as nn
import torch.nn.functional as F # 通常F被用于调用无参数的函数式API

class MyMLP(nn.Module):
    def __init__(self):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.view(-1, 784) # -1 会自动计算为 BatchSize (B)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x